In [141]:
import pandas as pd
import numpy as np
import os
import random
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

# Seed 고정 함수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

# CSV를 Parquet으로 변환하여 메모리 효율성 증대
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')

# 데이터 변환
# csv_to_parquet('./train.csv', 'train')
# csv_to_parquet('./test.csv', 'test')

# 데이터 로드
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [142]:
# Carrier_Code(IATA) → Airline 복구
airline_mapping = train[['Carrier_Code(IATA)', 'Airline']].dropna().drop_duplicates().set_index('Carrier_Code(IATA)')['Airline'].to_dict()
train['Airline'] = train['Carrier_Code(IATA)'].map(airline_mapping).fillna(train['Airline'])
test['Airline'] = test['Carrier_Code(IATA)'].map(airline_mapping).fillna(test['Airline'])

# 변환 결과 확인
print("Airline 복구 후 결측치 수:")
print("Train Airline 결측치:", train['Airline'].isna().sum())
print("Test Airline 결측치:", test['Airline'].isna().sum())

print("Airline Mapping Dictionary:")
print(pd.Series(airline_mapping))


Airline 복구 후 결측치 수:
Train Airline 결측치: 11864
Test Airline 결측치: 26893
Airline Mapping Dictionary:
WN      Southwest Airlines Co.
UA                    Cape Air
AA       Trans States Airlines
DL    ExpressJet Airlines Inc.
AS      Peninsula Airways Inc.
B6             JetBlue Airways
NK            Spirit Air Lines
F9      Frontier Airlines Inc.
HA        Empire Airlines Inc.
G4               Allegiant Air
VX              Virgin America
dtype: object


In [143]:
# Airline → Carrier_ID(DOT) 복구
carrier_mapping = train[['Airline', 'Carrier_ID(DOT)']].dropna().drop_duplicates()
carrier_mapping = carrier_mapping.groupby('Airline').first()  # Airline 기준으로 첫 번째 Carrier_ID(DOT)를 선택

# Carrier_ID(DOT) 복구
train['Carrier_ID(DOT)'] = train['Airline'].map(carrier_mapping['Carrier_ID(DOT)']).fillna(train['Carrier_ID(DOT)'])
test['Carrier_ID(DOT)'] = test['Airline'].map(carrier_mapping['Carrier_ID(DOT)']).fillna(test['Carrier_ID(DOT)'])

# 변환 결과 확인
print("Carrier_ID(DOT) 복구 후 결측치 수:")
print("Train Carrier_ID(DOT) 결측치:", train['Carrier_ID(DOT)'].isna().sum())
print("Test Carrier_ID(DOT) 결측치:", test['Carrier_ID(DOT)'].isna().sum())

print("Carrier Mapping Dictionary:")
print(carrier_mapping)

Carrier_ID(DOT) 복구 후 결측치 수:
Train Carrier_ID(DOT) 결측치: 1273
Test Carrier_ID(DOT) 결측치: 2933
Carrier Mapping Dictionary:
                                           Carrier_ID(DOT)
Airline                                                   
Air Wisconsin Airlines Corp                        20046.0
Alaska Airlines Inc.                               19930.0
Allegiant Air                                      20368.0
American Airlines Inc.                             19805.0
Cape Air                                           20304.0
Capital Cargo International                        20427.0
Comair Inc.                                        20397.0
Commutair Aka Champlain Enterprises, Inc.          20445.0
Compass Airlines                                   21167.0
Delta Air Lines Inc.                               19790.0
Empire Airlines Inc.                               19690.0
Endeavor Air Inc.                                  20363.0
Envoy Air                                          2039

In [144]:
# K-Means를 활용한 EDT/EAT 복구
features_for_kmeans = ['Origin_Airport_ID', 'Destination_Airport_ID', 'Month', 'Day_of_Month', 'Distance']

# KMeans 학습
kmeans_data = train.dropna(subset=['Estimated_Departure_Time', 'Estimated_Arrival_Time'])[features_for_kmeans].copy()
kmeans_target = train.dropna(subset=['Estimated_Departure_Time', 'Estimated_Arrival_Time']).copy()

kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(kmeans_data)

# 클러스터와 평균 비행 시간 계산
kmeans_target['Cluster'] = kmeans.predict(kmeans_data)
kmeans_target['Flight_Time'] = (
    kmeans_target['Estimated_Arrival_Time'] - kmeans_target['Estimated_Departure_Time']
) % 1440
cluster_time_mapping = kmeans_target.groupby('Cluster')['Flight_Time'].mean().to_dict()

# 복구 함수
def recover_time(row, col):
    if pd.isna(row['Estimated_Departure_Time']) or pd.isna(row['Estimated_Arrival_Time']):
        # 예측 시에도 feature 이름을 포함한 DataFrame 전달
        cluster = kmeans.predict(pd.DataFrame([row[features_for_kmeans].values], columns=features_for_kmeans))[0]
        avg_time = cluster_time_mapping.get(cluster, np.nan)
        if col == 'Estimated_Departure_Time' and not pd.isna(row['Estimated_Arrival_Time']):
            return (row['Estimated_Arrival_Time'] - avg_time) % 1440
        elif col == 'Estimated_Arrival_Time' and not pd.isna(row['Estimated_Departure_Time']):
            return (row['Estimated_Departure_Time'] + avg_time) % 1440
    return row[col]

# EDT와 EAT 복구
for col in ['Estimated_Departure_Time', 'Estimated_Arrival_Time']:
    train[col] = train.apply(lambda row: recover_time(row, col), axis=1)
    test[col] = test.apply(lambda row: recover_time(row, col), axis=1)


In [145]:
# 불필요한 특성 제거
columns_to_drop = ['Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train = train.drop(columns=columns_to_drop, errors='ignore')
test = test.drop(columns=columns_to_drop, errors='ignore')

In [146]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  988312 non-null   float64
 4   Estimated_Arrival_Time    988312 non-null   float64
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Destination_Airport_ID    1000000 non-null  int64  
 7   Distance                  1000000 non-null  float64
 8   Carrier_ID(DOT)           998727 non-null   float64
 9   Tail_Number               1000000 non-null  object 
 10  Delay                     255001 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 83.9+ MB


In [147]:
# 남은 결측치 처리
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns
train[numeric_cols] = train[numeric_cols].fillna(train[numeric_cols].mean())
test[numeric_cols] = test[numeric_cols].fillna(test[numeric_cols].mean())

categorical_cols = train.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col in test.columns:
        mode = train[col].mode()[0]
        train[col] = train[col].fillna(mode)
        test[col] = test[col].fillna(mode)

In [152]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  1000000 non-null  float64
 4   Estimated_Arrival_Time    1000000 non-null  float64
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Destination_Airport_ID    1000000 non-null  int64  
 7   Distance                  1000000 non-null  float64
 8   Carrier_ID(DOT)           1000000 non-null  float64
 9   Tail_Number               1000000 non-null  object 
 10  Delay                     255001 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 83.9+ MB


In [136]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  1000000 non-null  float64
 4   Estimated_Arrival_Time    1000000 non-null  float64
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Destination_Airport_ID    1000000 non-null  int64  
 7   Distance                  1000000 non-null  float64
 8   Carrier_ID(DOT)           1000000 non-null  float64
 9   Tail_Number               1000000 non-null  object 
 10  Delay                     489332 non-null   object 
 11  Flight_Time               1000000 non-null  float64
dtypes: float64(5), int64(4), object(3)
memory usage: 91.6+ MB


In [111]:
# 테스트 데이터 예측 및 제출 파일 생성

# object 타입 열을 category로 변환
for col in test.select_dtypes(include='object').columns:
    test[col] = test[col].astype('category')

# 테스트 데이터 예측
test_probabilities = final_model.predict_proba(test)

# 제출 파일 생성
submission = pd.DataFrame({
    'ID': test['ID'],  # 테스트 데이터의 ID 열 사용
    'Not_Delayed': test_probabilities[:, 0],
    'Delayed': test_probabilities[:, 1]
})
submission.to_csv('submission.csv', index=False)
print("제출 파일 생성 완료: submission.csv")

제출 파일 생성 완료: submission.csv
